In [1]:
# GENERAL IMPORTS

import pandas as pd
import numpy as np
import scipy as sp

import plotly.offline as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='lnumerow', api_key='mDhzegrhH0qf8rlFbzOo')
plotly.offline.init_notebook_mode(connected=True)

import datetime
import math

In [5]:
datafile = r'''Data\IVE_bidask1min.txt'''

colnames = ['Date', 'Time', 'BidOpen', 'BidHigh', 'BidLow', 'BidClose', 'AskOpen', 'AskLow', 'AskHigh', 'AskClose']
fullpricedata = pd.read_csv(datafile, names=colnames)

fullpricedata['DateTime'] = (fullpricedata['Date']+fullpricedata['Time']).map(lambda x: datetime.datetime(int(x[6:10]), int(x[0:2]), int(x[3:5]), int(x[10:12]), int(x[13:15])))
del fullpricedata['Date']
del fullpricedata['Time']

fullpricedata = fullpricedata[[(dt >= datetime.datetime(dt.year, dt.month, dt. day, 9, 30, 0)) and (dt <= datetime.datetime(dt.year, dt.month, dt. day, 16, 0, 0)) for dt in fullpricedata['DateTime']]].copy()

In [12]:
pricedata = fullpricedata[fullpricedata['DateTime'] > datetime.datetime(2019,2,12,0,0,0)].copy()

In [13]:
def candlestickChart():
    
    data = [ dict(
            type = 'candlestick',
            x = [ii for ii in range(len(pricedata['BidLow']))],
            low = pricedata['BidLow'],
            open = pricedata['BidOpen'],
            close = pricedata['BidClose'],
            high = pricedata['BidHigh'])
           ]

    layout = dict(
            title = 'Candlestick Chart',
            xaxis = dict(
                title = dict(
                    text = 'Time'
                )
            ),
            yaxis = dict(
                title = dict(
                    text = 'Price'
                )
            ))
        
    return go.FigureWidget( data=data, layout=layout)

In [14]:
candle = candlestickChart()
candle

FigureWidget({
    'data': [{'close': array([110.42, 110.47, 110.48, ..., 111.08, 111.11, 111.11]),
          …